Support Vector Machines (SVM) adalah model Machine Learning yang serbaguna dan kuat, mampu melakukan klasifikasi linear atau nonlinear, regresi, dan deteksi outlier. SVM sangat cocok untuk klasifikasi dataset kompleks berukuran kecil atau menengah.

**1. Klasifikasi SVM Linear**
Ide dasar di balik SVM adalah menemukan "jalan" terlebar yang mungkin (margin) antara kelas-kelas yang berbeda. Ini disebut klasifikasi margin besar (*large margin classification*).

* **Large Margin Classification**: SVM classifier tidak hanya memisahkan dua kelas tetapi juga menjaga jarak sejauh mungkin dari *training instance* terdekat. *Instance* yang berada di tepi "jalan" ini disebut *support vector*.
* **Sensitivitas terhadap Skala Fitur**: SVM sensitif terhadap skala fitur. Penskalaan fitur (misalnya, menggunakan `StandardScaler` dari Scikit-Learn) dapat meningkatkan *decision boundary* secara signifikan.
* **Hard Margin vs. Soft Margin Classification**:
    * **Hard Margin Classification**: Mensyaratkan semua *instance* berada di luar "jalan" dan di sisi yang benar. Masalah utamanya adalah hanya berfungsi jika data dapat dipisahkan secara linear dan sangat sensitif terhadap outlier.
    * **Soft Margin Classification**: Model yang lebih fleksibel yang bertujuan untuk menyeimbangkan ukuran "jalan" (margin) dengan jumlah *margin violation* (*instance* yang berada di tengah "jalan" atau di sisi yang salah). Hiperparameter `C` mengontrol *trade-off* ini. Nilai `C` yang rendah menghasilkan margin yang lebih besar tetapi dengan lebih banyak *margin violation*, yang cenderung menggeneralisasi lebih baik jika model *overfitting*.

**Contoh Kode Linear SVM Classifier (Scikit-Learn)**:

```python
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2, 3)] # petal length, petal width
y = (iris["target"] == 2).astype(np.float64) # Iris virginica

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])
svm_clf.fit(X, y)

# Contoh prediksi
print(svm_clf.predict([[5.5, 1.7]])) # Output: array([1.])
```

Kode ini memuat dataset iris, menskalakan fitur, dan melatih model SVM linear menggunakan kelas `LinearSVC` untuk mendeteksi bunga Iris virginica.

* `LinearSVC` tidak menghasilkan probabilitas kelas.
* Alternatif `LinearSVC` adalah kelas `SVC` dengan `kernel="linear"` atau `SGDClassifier` dengan `loss="hinge"`. `SGDClassifier` berguna untuk tugas klasifikasi *online* atau dataset besar.
* Penting untuk menskalakan data saat menggunakan `LinearSVC` karena ia meregulasi bias term. Juga, pastikan `loss` disetel ke "hinge" dan `dual` disetel ke `False` (kecuali jika fitur lebih banyak dari *instance* pelatihan).

**2. Klasifikasi SVM Nonlinear**
Ketika dataset tidak dapat dipisahkan secara linear, ada dua pendekatan utama:

* **Menambahkan Fitur Polinomial**: Menambahkan fitur baru, seperti fitur polinomial, dapat membuat dataset terpisah secara linear.
    * **Contoh Kode (PolynomialFeatures dengan LinearSVC)**:

    ```python
    from sklearn.datasets import make_moons
    from sklearn.pipeline import Pipeline
    from sklearn.preprocessing import PolynomialFeatures
    from sklearn.svm import LinearSVC

    X, y = make_moons(n_samples=100, noise=0.15)
    polynomial_svm_clf = Pipeline([
        ("poly_features", PolynomialFeatures(degree=3)),
        ("scaler", StandardScaler()),
        ("svm_clf", LinearSVC(C=10, loss="hinge"))
    ])
    polynomial_svm_clf.fit(X, y)
    ```

    Pendekatan ini dapat bekerja dengan baik tetapi menjadi lambat dengan derajat polinomial yang tinggi karena menciptakan banyak fitur.

* **Kernel Trick**: Teknik matematis yang memungkinkan SVM untuk mencapai hasil yang sama seolah-olah Anda telah menambahkan banyak fitur polinomial (bahkan dengan derajat sangat tinggi) tanpa benar-benar menambahkannya. Ini mencegah ledakan kombinatorial jumlah fitur.
    * **Polynomial Kernel**: Diimplementasikan oleh kelas `SVC` dengan `kernel="poly"`.
        * `degree`: Mengontrol derajat polinomial. Kurangi jika *overfitting*, tingkatkan jika *underfitting*.
        * `coef0`: Mengontrol seberapa besar model dipengaruhi oleh polinomial derajat tinggi versus derajat rendah.
    * **Similarity Features (Gaussian RBF Kernel)**: Menambahkan fitur yang dihitung menggunakan fungsi kemiripan, seperti Gaussian Radial Basis Function (RBF).
        * Rumus Gaussian RBF: $\phi_{\gamma}(x,l)=exp(-\gamma||x-l||^{2})$.
        * Kernel RBF Gaussian memungkinkan untuk mendapatkan hasil yang mirip seolah-olah Anda menambahkan banyak fitur kemiripan.
    * **Contoh Kode (SVC dengan Gaussian RBF Kernel)**:

    ```python
    from sklearn.svm import SVC

    # X dan y dari make_moons() sebelumnya
    rbf_kernel_svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
    ])
    rbf_kernel_svm_clf.fit(X, y)
    ```

    * `gamma` ($\gamma$): Bertindak sebagai hiperparameter regularisasi. Meningkatkan `gamma` membuat kurva berbentuk lonceng lebih sempit, mengurangi jangkauan pengaruh setiap *instance*, menghasilkan *decision boundary* yang lebih tidak teratur (cenderung *overfitting*). Mengurangi `gamma` membuat kurva lebih lebar, *instance* memiliki jangkauan pengaruh yang lebih besar, dan *decision boundary* menjadi lebih halus (cenderung *underfitting*).

**Memilih Kernel**:
* Selalu coba *linear kernel* terlebih dahulu (`LinearSVC` lebih cepat dari `SVC(kernel="linear")`), terutama jika dataset pelatihan sangat besar atau memiliki banyak fitur.
* Jika dataset pelatihan tidak terlalu besar, coba *Gaussian RBF kernel* karena seringkali bekerja dengan baik.
* Bereksperimenlah dengan *cross-validation* dan *grid search* untuk kernel lainnya, terutama jika ada kernel yang khusus untuk struktur data Anda.

**3. Kompleksitas Komputasi**
* **`LinearSVC`**: Berbasis library `liblinear`. Tidak mendukung *kernel trick*. Skalanya hampir linear dengan jumlah *training instance* ($m$) dan jumlah fitur ($n$), dengan kompleksitas waktu pelatihan sekitar $O(m \times n)$. Sensitif terhadap *tolerance hyperparameter* ($\epsilon$).
* **`SVC`**: Berbasis library `libsvm`. Mendukung *kernel trick*. Kompleksitas waktu pelatihan biasanya antara $O(m^2 \times n)$ dan $O(m^3 \times n)$. Menjadi sangat lambat jika jumlah *training instance* besar. Cocok untuk dataset pelatihan kompleks berukuran kecil atau menengah. Skala baik dengan jumlah fitur, terutama fitur *sparse*.
* **`SGDClassifier`**: Skalanya $O(m \times n)$ dan mendukung *out-of-core training* (dapat menangani dataset yang tidak muat di memori).

**4. Regresi SVM**
Algoritma SVM juga mendukung regresi linear dan nonlinear.
* Tujuannya adalah untuk menyesuaikan *instance* sebanyak mungkin di dalam "jalan" sambil membatasi *margin violation*.
* Lebar "jalan" dikontrol oleh hiperparameter $\epsilon$. Model dikatakan $\epsilon$-insensitive karena menambahkan lebih banyak *training instance* di dalam margin tidak memengaruhi prediksi model.
* **`LinearSVR`**: Digunakan untuk regresi SVM linear.
* **`SVR`**: Digunakan untuk regresi SVM *kernelized* (nonlinear).

**Contoh Kode Linear SVM Regression (Scikit-Learn)**:

```python
from sklearn.svm import LinearSVR

# Asumsikan X dan y sudah diskalakan dan terpusat
svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)
```

**Contoh Kode Nonlinear SVM Regression (Scikit-Learn)**:

```python
from sklearn.svm import SVR

# Asumsikan X dan y
svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)
```

**5. Di Balik Layar (Teoritis)**
* **Fungsi Keputusan (*Decision Function*)**: Untuk SVM classifier linear, kelas dari *instance* baru $x$ diprediksi dengan menghitung fungsi keputusan $w^{\top}x+b$. Jika hasilnya positif, kelas yang diprediksi adalah kelas positif (1), jika tidak, itu adalah kelas negatif (0).
    * *Decision boundary* adalah kumpulan titik di mana fungsi keputusan sama dengan 0.
    * *Dashed line* pada plot menunjukkan titik-titik di mana fungsi keputusan sama dengan 1 atau -1: mereka paralel dan berjarak sama dari *decision boundary*, membentuk margin di sekitarnya.
* **Tujuan Pelatihan (*Training Objective*)**:
    * Tujuan utama adalah meminimalkan $||w||$ (norma vektor bobot) untuk mendapatkan margin yang lebih besar.
    * **Hard Margin Linear SVM Classifier Objective**: Meminimalkan $\frac{1}{2}w^{\top}w$ (setara dengan meminimalkan $\frac{1}{2}||w||^2$, yang memiliki turunan yang lebih sederhana $w$). Dibatasi oleh $t^{(i)}(w^{\top}x^{(i)}+b)\ge1$ untuk semua *instance* (di mana $t^{(i)}$ adalah -1 untuk *instance* negatif dan 1 untuk *instance* positif).
    * **Soft Margin Linear SVM Classifier Objective**: Memperkenalkan *slack variable* $\zeta^{(i)}\ge0$ untuk setiap *instance* yang mengukur seberapa banyak *instance* ke-i diizinkan melanggar margin. Tujuannya adalah meminimalkan $\frac{1}{2}w^{\top}w+C\sum_{i=1}^{m}\zeta^{(i)}$ dengan batasan $t^{(i)}(w^{\top}x^{(i)}+b)\ge1-\zeta^{(i)}$ dan $\zeta^{(i)}\ge0$. `C` mengatur *trade-off* antara memperkecil *slack variable* dan memperbesar margin.
* **Quadratic Programming (QP)**: Masalah *hard margin* dan *soft margin* keduanya merupakan masalah optimisasi kuadrat cembung dengan batasan linear, dikenal sebagai masalah *Quadratic Programming*. QP *solver* dapat digunakan untuk menyelesaikan masalah ini.
* **Masalah Dual (*Dual Problem*)**: SVM dapat diselesaikan dengan memecahkan masalah primal atau masalah dualnya, karena keduanya memiliki solusi yang sama di bawah kondisi tertentu. Masalah dual lebih cepat diselesaikan ketika jumlah *training instance* lebih kecil dari jumlah fitur. Yang lebih penting, masalah dual memungkinkan *kernel trick*.
    * **Bentuk Dual dari Linear SVM Objective**:
        $\text{minimize} \frac{1}{2}\sum_{i=1}^{m}\sum_{j=1}^{m}\alpha^{(i)}\alpha^{(j)}t^{(i)}t^{(j)}x^{(i)\tau}x^{(j)}-\sum_{i=1}^{m}\alpha^{(i)}$
        subject to $\alpha^{(i)}\ge0$ for $i=1,2,\cdot\cdot\cdot,m$
    * Setelah menemukan $\hat{\alpha}$ yang meminimalkan persamaan ini, $\hat{w}$ dan $\hat{b}$ dapat dihitung menggunakan persamaan di Equation 5-7.
* **Kernelized SVMs**:
    * **Tantangan Nonlinearitas**: Ketika data tidak dapat dipisahkan secara linear di ruang fitur asli, salah satu solusinya adalah memetakan data ke ruang fitur dimensi yang lebih tinggi di mana data mungkin menjadi terpisah secara linear. Namun, pemetaan eksplisit ke ruang dimensi tinggi dapat sangat mahal secara komputasi dan bahkan tidak mungkin jika dimensinya tak terbatas.
    * **Solusi Kernel Trick**: *Kernel trick* adalah teknik brilian yang menghindari kebutuhan untuk secara eksplisit melakukan transformasi ini. Sebaliknya, ia menggantikan *dot product* di ruang fitur dimensi tinggi dengan fungsi *kernel* yang dapat dihitung langsung dari vektor di ruang fitur asli.
        * Ini bekerja karena algoritma SVM, dalam bentuk dualnya, hanya bergantung pada *dot product* antara *training instance*. Dengan mengganti *dot product* ini dengan fungsi *kernel*, kita secara efektif melatih SVM di ruang fitur dimensi tinggi tanpa pernah benar-benar bekerja di ruang itu.
    * **Contoh Kernel**:
        * **Polynomial Kernel**: $K(a,b)=(ya^{\top}b+r)^{d}$. Ini cocok untuk dataset yang dapat dipisahkan secara linear setelah transformasi polinomial.
        * **Gaussian RBF Kernel**: $K(a,b)=exp(-\gamma||a-b||^{2})$. Kernel ini secara implisit memetakan *instance* ke ruang dimensi tak terbatas, sangat efektif untuk menangani hubungan nonlinear yang kompleks. Parameter `gamma` mengontrol "luas" jangkauan pengaruh setiap *instance*.
* **Online SVMs**: Untuk SVM classifier linear, salah satu metode untuk mengimplementasikan SVM *online* adalah menggunakan *Gradient Descent* (misalnya, dengan `SGDClassifier`) untuk meminimalkan fungsi biaya.
    * **Fungsi Biaya Linear SVM Classifier**:
        $I(w,b)=\frac{1}{2}w^{T}w+C\sum_{i=1}^{m}max(0,1-t^{(i)}(w^{T}x^{(i)}+b))$
        * Suku pertama (`$\frac{1}{2}w^{T}w$`) mendorong model untuk memiliki vektor bobot `w` yang kecil (margin yang lebih besar).
        * Suku kedua (`$C\sum_{i=1}^{m}max(0,1-t^{(i)}(w^{T}x^{(i)}+b))$`) menghitung total *margin violation* dan meminimalkannya.
    * **Hinge Loss**: Fungsi $max(0,1-t)$ disebut fungsi *hinge loss*.

Secara keseluruhan, SVM adalah algoritma yang canggih yang memanfaatkan konsep geometris margin maksimal dan teknik matematika cerdas seperti *kernel trick* dan dualitas untuk menangani masalah klasifikasi dan regresi, baik linear maupun nonlinear, secara efisien.